In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
from cot_reports import cot_all_reports
cot_data_tuple = cot_all_reports(store_txt=True, verbose=True)
legacy_fut = cot_data_tuple[0]

legacy_fut
Selected: COT Legacy report. Futures only.
Stored the extracted file FUT86_16.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2017
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2018
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2019
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2020
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2021
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2022
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2023
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2024
Stored the file annual.txt in 

In [3]:
from utils.data_loader import load_portfolio_data

forex_data = load_portfolio_data(category="forex", years = 20, legacy_fut= legacy_fut)

⏳ Loading data for EURUSD…
Loading GDP_Growth → ECONOMICS:EUGDPYY (3M)
Loading Interest_Rate → ECONOMICS:EUINTR (1M)
Loading Inflation_Rate → ECONOMICS:EUIRYY (1M)
Loading CPI → ECONOMICS:EUCPI (1M)
Loading PPI → ECONOMICS:EUPPI (1M)
Loading Unemployment → ECONOMICS:EUUR (1M)
Loading Trade_Balance → ECONOMICS:EUBOT (1M)
Loading Gov_Debt → ECONOMICS:EUGDG (12M)
Loading Consumer_Confidence → ECONOMICS:EUCCI (1M)
Loading Retail_Sales → ECONOMICS:EURSMM (1M)
Loading Money_Supply → ECONOMICS:EUM2 (1M)
Latest FX rate (EURUSD) 2025-07-24: 1.17408
⏳ Loading data for GBPUSD…
Loading GDP_Growth → ECONOMICS:GBGDPYY (3M)
Loading Interest_Rate → ECONOMICS:GBINTR (1M)
Loading Inflation_Rate → ECONOMICS:GBIRYY (1M)
Loading CPI → ECONOMICS:GBCPI (1M)
Loading PPI → ECONOMICS:GBPPI (1M)
Loading Unemployment → ECONOMICS:GBUR (1M)
Loading Trade_Balance → ECONOMICS:GBBOT (1M)
Loading Gov_Debt → ECONOMICS:GBGDG (12M)
Loading Consumer_Confidence → ECONOMICS:GBCCI (1M)
Loading Retail_Sales → ECONOMICS:GBRSMM 

In [5]:
import pickle

with open("legacy_fut.pkl", "wb") as f:
    pickle.dump(legacy_fut, f)
# Lưu dữ liệu vào file
with open("forex_data.pkl", "wb") as f:
    pickle.dump(forex_data, f)

In [7]:
import sys
import os
import pickle
sys.path.append(os.path.abspath("../src"))

with open("legacy_fut.pkl", "rb") as f:
    legacy_fut = pickle.load(f)

with open("forex_data.pkl", "rb") as f:
    forex_data = pickle.load(f)

In [9]:

import numpy as np

def z_score(series):
    """Chuẩn hóa dữ liệu thành z-score."""
    mean = np.mean(series)
    std = np.std(series)
    if std == 0:
        return np.zeros_like(series)
    return (series - mean) / std

def macro_score(country_data: dict, weights: dict) -> float:
    """
    Tính điểm tổng hợp sức mạnh vĩ mô cho một quốc gia.
    
    Args:
        country_data: dict chứa các chỉ số macro đã chuẩn hóa.
        weights: dict trọng số từng chỉ số.

    Returns:
        float: điểm macro tổng hợp.
    """
    score = 0.0
    for key, weight in weights.items():
        value = country_data.get(key, 0)
        score += weight * value
    return score

def macro_score_diff(data_domestic: dict, data_foreign: dict, weights: dict) -> float:
    """
    Tính điểm chênh lệch sức mạnh macro giữa 2 quốc gia.
    
    Args:
        data_domestic: dict chỉ số quốc gia trong nước.
        data_foreign: dict chỉ số quốc gia đối tác.
        weights: dict trọng số.

    Returns:
        float: chênh lệch điểm (domestic - foreign).
    """
    score_dom = macro_score(data_domestic, weights)
    score_for = macro_score(data_foreign, weights)
    return score_dom - score_for

def fair_value_by_macro(base_fx_rate: float, score_diff: float, sensitivity: float = 0.02) -> float:
    """
    Ước lượng tỷ giá hợp lý dựa trên chênh lệch điểm macro.
    
    Args:
        base_fx_rate: tỷ giá hiện tại (spot).
        score_diff: chênh lệch điểm macro (domestic - foreign).
        sensitivity: hệ số nhạy cảm của tỷ giá theo điểm macro.

    Returns:
        float: tỷ giá hợp lý dự kiến.
    """
    return base_fx_rate * (1 + sensitivity * score_diff)

def is_overvalued(spot: float, fair_value: float, threshold: float = 0.01) -> bool:
    """
    Kiểm tra tiền tệ có đang bị định giá quá cao không.
    """
    deviation = (spot - fair_value) / fair_value
    return deviation > threshold

def is_undervalued(spot: float, fair_value: float, threshold: float = 0.01) -> bool:
    """
    Kiểm tra tiền tệ có đang bị định giá quá thấp không.
    """
    deviation = (spot - fair_value) / fair_value
    return deviation < -threshold

def generate_signal(spot: float, fair_value: float, threshold: float = 0.01) -> str:
    """
    Sinh tín hiệu giao dịch đơn giản dựa trên mức định giá.
    
    Returns:
        "BUY", "SELL", hoặc "HOLD"
    """
    if is_undervalued(spot, fair_value, threshold):
        return "BUY"
    elif is_overvalued(spot, fair_value, threshold):
        return "SELL"
    else:
        return "HOLD"

def extract_latest_close(data_dict):
    """
    Lấy giá trị close mới nhất từ dict chứa DataFrame.
    """
    result = {}
    for key, df in data_dict.items():
        if not df.empty:
            result[key] = df['close'].iloc[-1]
    return result

def normalize_macro_data(data1: dict, data2: dict):
    """
    Chuẩn hóa 2 dict macro thành z-score.
    """
    all_keys = set(data1.keys()).union(data2.keys())
    result1, result2 = {}, {}
    for key in all_keys:
        val1 = data1.get(key, 0)
        val2 = data2.get(key, 0)
        values = np.array([val1, val2])
        z = z_score(values)
        result1[key] = z[0]
        result2[key] = z[1]
    return result1, result2


In [12]:
forex_data['EURUSD']['economic_data']
forex_data['DX1!']['economic_data']

{'GDP_Growth':                        time  open  high  low  close
 0 2025-01-01 00:00:00+00:00   2.0   2.0  2.0    2.0,
 'Interest_Rate':                        time  open  high  low  close
 0 2025-06-01 00:00:00+00:00   4.5   4.5  4.5    4.5,
 'Inflation_Rate':                        time  open  high  low  close
 0 2025-06-01 00:00:00+00:00   2.7   2.7  2.7    2.7,
 'CPI':                        time    open    high     low   close
 0 2025-06-01 00:00:00+00:00  322.56  322.56  322.56  322.56,
 'PPI':                        time    open    high     low   close
 0 2025-06-01 00:00:00+00:00  148.24  148.24  148.24  148.24,
 'Unemployment':                        time  open  high  low  close
 0 2025-06-01 00:00:00+00:00   4.1   4.1  4.1    4.1,
 'Trade_Balance':                        time          open          high           low  \
 0 2025-05-01 00:00:00+00:00 -7.152000e+10 -7.152000e+10 -7.152000e+10   
 
           close  
 0 -7.152000e+10  ,
 'Gov_Debt':                        time 

In [ ]:
# 1. Lấy macro close mới nhất
eur_macro = extract_latest_close(forex_data['EURUSD']['economic_data'])
usd_macro = extract_latest_close(forex_data['DX1!']['economic_data'])

# 2. Trọng số từng chỉ số
weights = {
    "GDP_Growth": 0.2,                # Tăng trưởng cao → mạnh
    "Interest_Rate": 0.25,            # Lãi suất cao → hút vốn → mạnh
    "Inflation_Rate": -0.2,           # Lạm phát cao → xấu
    "Unemployment": -0.1,             # Thất nghiệp cao → xấu
    "Gov_Debt": -0.15,                # Nợ công cao → rủi ro
    "Consumer_Confidence": 0.15,      # Niềm tin tăng → tốt
    "Retail_Sales": 0.1,              # Bán lẻ tăng → cầu nội địa tốt → hỗ trợ tiền tệ
    "Money_Supply": -0.1,             # Cung tiền cao → dễ lạm phát → xấu
    "Trade_Balance": 0.1,             # Thặng dư thương mại → tích cực
    "CPI": -0.1,                      # CPI cao → tương tự Inflation_Rate
    "PPI": -0.05                      # Giá sản xuất cao → dễ lạm phát → tiêu cực
}

# 3. Chuẩn hóa
eur_z, usd_z = normalize_macro_data(eur_macro, usd_macro)

# 4. Tính macro chênh lệch
score_diff = macro_score_diff(eur_z, usd_z, weights)

# 5. Tính fair value
spot = 1.085  # tỷ giá EURUSD hiện tại
fair_value = fair_value_by_macro(spot, score_diff)

# 6. Sinh tín hiệu
signal = generate_signal(spot, fair_value)

# 7. In kết quả
print("Spot:", spot)
print("Fair Value:", round(fair_value, 5))
print("Macro Score Diff:", round(score_diff, 3))
print("Signal:", signal)


Spot: 1.085
Fair Value: 1.06981
Macro Score Diff: -0.7
Signal: SELL
